In [1]:
import pandas as pd
import re

from datetime import date, timedelta
import time

import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [2]:
def get_estate_news():
    
    
    # 부동산 뉴스 기본 url
    url_base = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=260&sid1=101&date='
    
    # 기사 작성일 기준 전날부터 일주일을 분석하기 위해 날짜 생성
    dates = []
    for i in range(7):
        dates.append((date.today()- timedelta(days = i+1)).strftime("%Y%m%d"))
    
    
    # 기사 크롤러 시작   :  일자별(7일), 페이지별(1 ~ 마지막페이지), 각 기사별 크롤링. 순서는 역순.
    title, content = [], []
    for dt in dates:   # 7일 반복
        first_url = url_base + dt + '&page=1' # 첫번째 일자에 첫 페이지 읽기
        first_html = urlopen(first_url).read()  # xml로 파싱하기 위함
        first_soup = bs(first_html, 'html.parser') # 페이지 파싱

        first_pages = first_soup.find('div',{'class':'paging'})  # 하단의 페이지 읽기 ex) '1', '2', '3' ...
        if first_pages.text[-3:-1] =='다음': # 10 페이지 이상일 경우 다음을 눌러야 11페이지가 나옴.
            first_page_max = 10 # 다음이 있을땐 현재 페이지에선 10페이지가 마지막
            
            for page in range(1,first_page_max+1): # 10페이지가 넘어갈 경우 반복하여 1~10페이지 파싱
                url = url_base + dt + '&page=' + str(page) # 각 페이지별 url 주소 입력
                html = urlopen(url).read() # 각 페이지별 url 읽기.
                xmlsoup = bs(html, 'html.parser') # 파싱

                
                lis = xmlsoup.select('#main_content > div.list_body.newsflash_body > \
                                    ul.type06_headline > li') # 기사 list
                
                for i in lis: # list 내 기사 링크로 이동하여 제목과 내용 크롤링
                    link = i.find('a', attrs = {'href':re.compile('^https://')}).get('href')
                    news = urlopen(link).read()
                    newssoup = bs(news, 'html.parser')

                    title.append(newssoup.select('#articleTitle')[0].text)
                    content.append(newssoup.select('#articleBodyContents')[0].text)
            
            # 10페이지 이후 다음페이지로 넘어가서 11페이지부터 위 작업 반복하기
            second_url = 'https://news.naver.com/main/list.nhn' + first_pages.find('a',{'class':'next'}).get('href')
            second_html = urlopen(second_url).read()
            second_soup = bs(second_html, 'html.parser')
            
            second_pages = second_soup.find('div',{'class':'paging'})  # 하단의 페이지 읽기 ex) '1', '2', '3' ...
            if second_pages.text[-3:-1] =='다음': # 11-20에서 다음이 존재할 경우 (21p) 다음으로 넘어가기 위함
                second_page_max = 20 # 다음이 있을땐 현재 페이지에선 20페이지가 마지막

                for page in range(11,second_page_max+1):
                    url = url_base + dt + '&page=' + str(page)
                    html = urlopen(url).read()
                    xmlsoup = bs(html, 'html.parser') 

                    lis = xmlsoup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li')

                    for i in lis:
                        link = i.find('a', attrs = {'href':re.compile('^https://')}).get('href')
                        news = urlopen(link).read()
                        newssoup = bs(news, 'html.parser')

                        title.append(newssoup.select('#articleTitle')[0].text)
                        content.append(newssoup.select('#articleBodyContents')[0].text)
                
                third_url = 'https://news.naver.com/main/list.nhn' + second_pages.find('a',{'class':'next'}).get('href')
                third_html = urlopen(third_url).read()
                third_soup = bs(third_html, 'html.parser')
            
                third_pages = third_soup.find('div',{'class':'paging'})  # 하단의 페이지 읽기 ex) '1', '2', '3' ...
                try:       
                    third_page_max = int(third_pages.text[-3:-1])
                    for page in range(21,third_page_max+1):
                        url = url_base + dt + '&page=' + str(page)
                        html = urlopen(url).read()
                        xmlsoup = bs(html, 'html.parser') 

                        lis = xmlsoup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li')

                        for i in lis:
                            link = i.find('a', attrs = {'href':re.compile('^https://')}).get('href')
                            news = urlopen(link).read()
                            newssoup = bs(news, 'html.parser')

                            title.append(newssoup.select('#articleTitle')[0].text)
                            content.append(newssoup.select('#articleBodyContents')[0].text)
                except:
                    print('{} 날짜는 31페이지 이상이 존재합니다'.format(dt))
                
            else:
                second_page_max = int(second_pages.text[-3:-1])
                
                for page in range(11,second_page_max+1):
                    url = url_base + dt + '&page=' + str(page)
                    html = urlopen(url).read()
                    xmlsoup = bs(html, 'html.parser') 

                    lis = xmlsoup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li')

                    for i in lis:
                        link = i.find('a', attrs = {'href':re.compile('^https://')}).get('href')
                        news = urlopen(link).read()
                        newssoup = bs(news, 'html.parser')

                        title.append(newssoup.select('#articleTitle')[0].text)
                        content.append(newssoup.select('#articleBodyContents')[0].text)

        else: 
            first_page_max = int(first_pages.text[-2])

            
        for page in range(1,first_page_max+1):
            url = url_base + dt + '&page=' + str(page)
            html = urlopen(url).read()
            xmlsoup = bs(html, 'html.parser') 

            lis = xmlsoup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li')

            for i in lis:
                link = i.find('a', attrs = {'href':re.compile('^https://')}).get('href')
                news = urlopen(link).read()
                newssoup = bs(news, 'html.parser')

                title.append(newssoup.select('#articleTitle')[0].text)
                content.append(newssoup.select('#articleBodyContents')[0].text)
        

    df_news = pd.DataFrame({'title':title, 'content':content})
    df_news['content'] = df_news['content'].apply(lambda x : ' '.join(x.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "").split()))
    
    return df_news

In [3]:
start = time.time()

df_news = get_estate_news()

end = time.time() - start
print(end)

327.16304898262024


In [140]:
df_news['content'] = df_news['content'].apply(lambda x : ' '.join(x.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "").split()))

In [142]:
df_news

,title,content
0,서울 아파트 지역별 온도차 뚜렷… 강남권 매수심리 위축,[파이낸셜뉴스] 고가주택에 대한 대출규제를 담은 12.16대책 이후 서울 아파트 시...
1,12·16대책 이후 훈풍 분다던 오피스텔...작년 대비 거래량 9.9% 감소해,서울 송파구 문정동 오피스텔 전경. 자료=상가정보연구소 [파이낸셜뉴스] 고가 아파트...
2,[강신우의 하우쓱]“LH공공임대아파트 ‘로또분양’ 하나”,LH vs 입주민 분양가 ‘갈등’LH “감정평가액 100%가 분양가”주민 “5년임대...
3,[2월 셋째 주] 코로나19 불안에도 분양물량 본격 풀린다,[파이낸셜뉴스] 2월 셋째 주에는 청약 작업 이관과 코로나19로 움츠렸던 부동산 시...
4,"롯데캐슬 현장, 새해부터 자발적 봉사활동",주안 캐슬&더샵 봉사단이 환경정화를 위해 쓰레기를 줍고 있다./ 사진=롯데건설 제공...
...,...,...
1421,[아파트 돋보기]공동주택 관리도 전문가가 필요하다①,경기도 파주 운정신도시 아파트 단지(사진=이데일리DB)[이데일리 김용운 기자] 우리...
1422,코로나사태 길어지면…항공·정유화학↓ 디스플레이↑,(자료=유진투자증권)[서울=뉴시스] 박주연 기자 = 중국을 강타한 신종 코로나 바이...
1423,`오매불망` 과천 지식정보타운 첫 분양…당첨자 선정 기준 보니,`과천 제이드 자이`(S9블록) 이달 공급해야 최소 청약거주기간 1년 적용청약통장 ...
1424,“이럴 줄 알았음 안 들어와” 청년 울리는 청년임대주택 [밀착취재],"文정부 공약 '청년 임대주택', 공급 확대 치중해 관리 소홀 비판 / 입주민 ""쓰레..."
